# Learning Dynamics - Zero Initialization

In [1]:
import random

import numpy as np
import torch
from torch import Generator

from data import get_dataloader, seed_worker
from models import MNISTFNNModel, MNISTCNNModel
from train import Trainer

In [2]:
R_SEED = 4240

torch.manual_seed(R_SEED)
random.seed(R_SEED)
np.random.seed(R_SEED)

# 1. MNIST dataset

### 1.1 FNN Model

In [3]:
generator = Generator()
generator.manual_seed(R_SEED)

train_loader = get_dataloader(train=True, batch_size=100, flatten=True, shuffle=False, num_workers=1,
                              worker_init_fn=seed_worker, generator=generator)

In [ ]:
model = MNISTFNNModel()

trainer = Trainer(model)
trainer.train(train_loader, 2)

In [4]:
model = MNISTFNNModel()
model.zero_initialization("uniform")

trainer = Trainer(model)
trainer.train(train_loader, 2)

Epoch 2/2: 100%|████████████████████████████████████████| 600/600 [00:50<00:00, 11.90it/s]


# 1.2 CNN Model

In [3]:
generator = Generator()
generator.manual_seed(R_SEED)

train_loader = get_dataloader(train=True, batch_size=100, flatten=False, shuffle=False, num_workers=1,
                              worker_init_fn=seed_worker, generator=generator)

In [ ]:
model = MNISTCNNModel()

trainer = Trainer(model)
trainer.train(train_loader, 2)

In [4]:
model = MNISTCNNModel()
model.zero_initialization("uniform")

trainer = Trainer(model)
trainer.train(train_loader, 2)

Epoch 2/2: 100%|████████████████████████████████████████| 600/600 [00:40<00:00, 14.85it/s]
